# DataJoint Workflow for Neuropixels Analysis with Kilosort

+ This notebook will describe the steps for interacting with the data processed with the workflow.

+ This workflow is assembled from 4 DataJoint elements:
     + [element-lab](https://github.com/datajoint/element-lab)
     + [element-animal](https://github.com/datajoint/element-animal)
     + [element-session](https://github.com/datajoint/element-session)
     + [element-array-ephys](https://github.com/datajoint/element-array-ephys)

+ DataJoint provides abundant functions to query and fetch data.  For a detailed tutorials, visit our [general tutorial site](https://playground.datajoint.io/)

## Load modules

In [ ]:
import datajoint as dj
import matplotlib.pyplot as plt
import numpy as np

from workflow.pipeline import lab, subject, session, probe, ephys

## Workflow architecture

In [ ]:
dj.Diagram(subject.Subject) + dj.Diagram(session.Session) + dj.Diagram(probe) + dj.Diagram(ephys)

## Experimental metadata

In [ ]:
subject.Subject()

In [ ]:
session_key = (session.Session & 'subject="subject5"' & 'session_id = 1').fetch1('KEY')

In [ ]:
session_key

## Recordings for a particular session from one or more probes.

In [ ]:
ephys.EphysRecording & session_key

## Spike-sorting results

In [ ]:
ephys.CuratedClustering.Unit & session_key

## Generate a raster plot for one probe insertion, one parameter set, and the 'good' units

In [ ]:
clustering_query = (ephys.CuratedClustering.Unit & \
                session_key & \
                'insertion_number = 2' & \
                'paramset_idx=0' & \
                'cluster_quality_label = "good"')

units, unit_spiketimes = clustering_query.fetch('unit', 'spike_times')
x = np.hstack(unit_spiketimes)
y = np.hstack([np.full_like(s, u) for u, s in zip(units, unit_spiketimes)])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(32, 16))
ax.plot(x, y, '|')
ax.set_xlabel('Time (s)');
ax.set_ylabel('Unit');

## Plot the waveform of a unit

In [ ]:
unit_key = (clustering_query & 'unit = 1').fetch1('KEY')
unit_data = (ephys.CuratedClustering.Unit * ephys.WaveformSet.PeakWaveform & unit_key).fetch1()

In [ ]:
sampling_rate = (ephys.EphysRecording & session_key & 'insertion_number = 2').fetch1('sampling_rate')/1000 # in kHz
plt.plot(np.r_[:unit_data['peak_electrode_waveform'].size] * 1/sampling_rate, unit_data['peak_electrode_waveform'])
plt.xlabel('Time (ms)');
plt.ylabel(r'Voltage ($\mu$V)');

## Summary
This notebook highlights the major tables in the workflow and visualize some of the processed results. 